# Project 1 - Team BAK

## Step 1 - Getting started

In [40]:
#Import some libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import datetime
from helpfulfun import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# import the datasets
train_list = np.genfromtxt("train.csv", dtype=None, delimiter=",", skip_header =1, unpack=True, encoding=None)
train = np.array(train_list)

test_list = np.genfromtxt("test.csv", dtype=None, delimiter=",", skip_header =1, unpack=True, encoding=None)
test = np.array(test_list)

In [3]:
x_te = test[2:].T

In [4]:
y = train[1]
x_tr = train[2:].T
print("x: ", x_tr.shape, " y: ", y.shape)

x:  (250000, 30)  y:  (250000,)


## Step 2 - Preprocessing

In [5]:
y_tr = np.where(y == "s",1,0)
print(y_tr.shape, " and y[:5]: ", y_tr[:5])

(250000,)  and y[:5]:  [1 0 0 0 0]


In [6]:
# delete features with more than 30% NaN values
x_tr_prep = np.delete(x_tr, 0, 1) # infer this one later
x_te_prep = np.delete(x_te, 0, 1) # infer this one later
for i in [4, 4, 4, 9, 18, 18, 18, 18, 18, 18, 18]:
    x_tr_prep = np.delete(x_tr_prep, i, 1)
    x_te_prep = np.delete(x_te_prep, i, 1)
x_tr_prep = x_tr_prep.astype(float)
x_te_prep = x_te_prep.astype(float)
x_tr_prep.shape

(250000, 18)

In [7]:
x, mean_x, std_x = standardize(x_tr_prep)
x_te, _, _ = standardize(x_te_prep)

In [8]:
x.shape, x_te.shape

((250000, 19), (568238, 19))

## Step 3 - Implement ML Methods

#### Linear regression using gradient descent

In [30]:
def mean_squared_error_gd(y, tx, initial_w, max_iters, gamma):
    """The Gradient Descent (GD) algorithm.
        
    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        initial_w: numpy array of shape=(2, ). The initial guess (or the initialization) for the model parameters
        max_iters: a scalar denoting the total number of iterations of GD
        gamma: a scalar denoting the stepsize
        
    Returns:
        losses: a list of length max_iters containing the loss value (scalar) for each iteration of GD
        ws: a list of length max_iters containing the model parameters as numpy arrays of shape (2, ), for each iteration of GD 
    """
    # Define parameters to store w and loss
    loss = 0
    w = initial_w
    for n_iter in range(max_iters):
        gradient = compute_gradient(y, tx, w)
        loss = compute_loss(y, tx, w)
        
        w = w - gamma * gradient
        
        # print w and loss
        #print("GD iter. {bi}/{ti}: loss={l}, w0={w0}, w1={w1}".format(
         #     bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return w, loss
    

#### Linear regression using stochastic gradient descent


In [44]:
def mean_squared_error_sgd(y, tx, initial_w, max_iters, gamma):
    """The Stochastic Gradient Descent algorithm (SGD).
            
    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        initial_w: numpy array of shape=(2, ). The initial guess (or the initialization) for the model parameters
        batch_size: a scalar denoting the number of data points in a mini-batch used for computing the stochastic gradient
        max_iters: a scalar denoting the total number of iterations of SGD
        gamma: a scalar denoting the stepsize
        
    Returns:
        losses: a list of length max_iters containing the loss value (scalar) for each iteration of SGD
        ws: a list of length max_iters containing the model parameters as numpy arrays of shape (2, ), for each iteration of SGD 
    """
    
    # Define parameters to store w and loss
    loss = 0
    w = initial_w
    batch_size = 1000
    
    for n_iter in range(max_iters):
        for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
            gradient = compute_gradient(minibatch_y, minibatch_tx, w)
            loss = compute_loss(y, tx, w)
            w = w - gamma * gradient

        #print("SGD iter. {bi}/{ti}: loss={l}, w0={w0}, w1={w1}".format(
         #     bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return w, loss
    

#### Build polimonial basis function which can be used with Least Squares and Ridge Regression

In [15]:
def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree.
    
    Args:
        x: numpy array of shape (N,), N is the number of samples.
        degree: integer.
        
    Returns:
        poly: numpy array of shape (N,d+1)
    """
    poly = np.ones((len(x),1))
    for j in range( 1, degree + 1):
        poly = np.c_[poly, np.power(x, j)]
    return poly

#### Least squares regression using normal equations

In [47]:
def least_squares(y, tx):
    opt_weights = np.linalg.solve(tx.T.dot(tx), tx.T.dot(y))
    e = y - tx.dot(opt_weights)
    mse = 1/(2*len(y)) * e.T.dot(e)
    return opt_weights, mse

In [32]:
def polynomial_regression_ls(y, x, degrees=[1, 3, 7, 12]):
    """Constructing the polynomial basis function expansion of the data,
       and then running least squares regression."""
    ws = []
    losses = []
    for ind, degree in enumerate(degrees):
        tx = build_poly(x, degree)
        w, loss = least_squares(y, tx)
        rmse = np.sqrt(2 * loss)
        
        ws.append(w)
        losses.append(rmse)

        #print("Processing {i}th experiment, degree={d}, rmse={loss}".format(
         #     i=ind + 1, d=degree, loss=rmse))
    ind = argmin(losses)
    return ws[ind], losses[ind]

#### Ridge regression using normal equations

In [18]:
def ridge_regression(y, tx, lambda_ ):
    aI = 2 * tx.shape[0] * lambda_ * np.identity(tx.shape[1])
    a = tx.T.dot(tx) + aI
    b = tx.T.dot(y)
    return np.linalg.solve(a, b)

In [33]:
def ridge_regression_demo(x, y, degree, ratio, seed):
    """ridge regression demo."""
    lambdas = np.logspace(-5, 0, 15)
    # TODO split and add test data
    
    tx_tr = build_poly(x_tr, degree)
    rmse_tr = []
    for ind, lambda_ in enumerate(lambdas):
        weight = ridge_regression(y_tr, tx_tr, lambda_)
        rmse_tr.append(compute_rmse(y_tr, tx_tr, weight))
        
        #print("proportion={p}, degree={d}, lambda={l:.3f}, Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(
         #      p=ratio, d=degree, l=lambda_, tr=rmse_tr[ind], te=rmse_te[ind]))
    ind = argmin(rmse_tr)
    return ws[ind], losses[ind]

#### Logistic regression using gradient descent or SGD (y ∈ {0, 1})

In [20]:
# calculating loss with sigmoid and using gradient descent
def logistic_regression(y, tx, w):
    loss = calculate_loss_lr(y, tx, w)
    gradient = calculate_gradient_lr(y, tx, w)
    hessian = calculate_hessian(y, tx, w)
    return loss, gradient, hessian

In [21]:
def logistic_regression_gradient_descent(y, x):
    # init parameters
    max_iter = 500
    threshold = 1e-8
    gamma = .5
    losses = []

    tx = x
    w = np.zeros((tx.shape[1],), dtype=float)

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and gradient descent on w.
        loss, gradient, hessian = logistic_regression(y, tx, w)
        w -= gamma * np.linalg.solve(hessian, gradient)
        
        if iter % 100 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    print("loss={l}".format(l=calculate_loss_lr(y_tr, tx, w)))
    
    return w, losses[-1]


#### Regularized Logistic Regression

In [22]:
def logistic_regression_regularized_gradient_descent(y, x):
    # init parameters
    max_iter = 2000
    gamma = 0.5
    lambda_ = 0.0005
    threshold = 1e-8
    losses = []

    w = np.zeros((tx.shape[1],), dtype=float)

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_penalized_gradient(y, tx, w, gamma, lambda_)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    print("loss={l}".format(l=calculate_loss(y, tx, w)))
    return w, losses[-1]

## Step 4 - Get Predictions

In [23]:
def get_predictions(x, best_w):
    preds = x.dot(best_w).reshape((x.shape[0],))
    y_te = np.where(preds < .5,-1,1)
    y_pred = np.c_[test[0], y_te]
    print(y_pred[0:5])
    y_pred = np.insert(y_pred, 0, ["Id", "Prediction"], axis=0)
    return y_pred

In [24]:
def savePredictions(pred, title="submission"):
    np.savetxt(title + ".csv", pred, fmt="%s", delimiter=",")

In [25]:
initial_w = np.zeros((x.shape[1],), dtype=float)

In [34]:
# Mean Squared Error Gradient Descent
w_gd, loss_gd = mean_squared_error_gd(y_tr, x, initial_w, max_iters=150, gamma=.005)
pred = get_predictions(x_te, w_gd)
print("MSE - GD Loss: ", loss_gd)

[['350000' '-1']
 ['350001' '-1']
 ['350002' '-1']
 ['350003' '-1']
 ['350004' '1']]
MSE- GD Loss:  0.11338331749684086


In [45]:
# Mean Squared Error Stochastic Gradient Descent
w_sgd, loss_sgd = mean_squared_error_sgd(y_tr, x, initial_w, max_iters=150, gamma=.005)
pred = get_predictions(x_te, w_sgd)
print("MSE - SGD Loss: ", loss_sgd)

[['350000' '-1']
 ['350001' '-1']
 ['350002' '-1']
 ['350003' '-1']
 ['350004' '1']]
MSE - SGD Loss:  0.11336016574552966


In [49]:
# Least Squares
w_ls, loss_ls = least_squares(y_tr, x)
#w_poly, loss_poly = polynomial_regression_ls(y_tr, x)
pred = get_predictions(x_te, w_ls)
print("MSE - LS Loss: ", loss_sgd)

[['350000' '-1']
 ['350001' '-1']
 ['350002' '-1']
 ['350003' '-1']
 ['350004' '-1']]
MSE - LS Loss:  0.11336016574552966


In [50]:
# Ridge Regression
# TODO: add split data
seed = 56
degree = 7
split_ratio = 0.5
#w_rr, loss_rr = ridge_regression_demo(x, y, degree, split_ratio, seed)
w_rr = ridge_regression(y_tr, x, .0005 )
pred_rr = get_predictions(x_te, w_rr)
#print("MSE - RR Loss: ", loss_rr)

[['350000' '-1']
 ['350001' '-1']
 ['350002' '-1']
 ['350003' '-1']
 ['350004' '-1']]


In [ ]:
# Logictic Regression Gradient Descent
w_logreg, loss_logreg = logistic_regression_gradient_descent(y_tr, x)
pred_log = get_predictions(x_te, w_logreg)
print("RMSE - LogRed Loss: ", loss_logreg)

In [ ]:
# Regularized Logistic Regression with GD
w_reglog, loss_reglog = logistic_regression_regularized_gradient_descent(y_tr, x)
pred_reglog = get_predictions(x_te, w_reglog)
